# Decision Tree Lab

In [22]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import arff
import pandas as pd
from IPython.core.display import display
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
import math

import pprint
import pdb

In [2]:
pp = pprint.PrettyPrinter(indent=4)

In [3]:
def print_encodings(enc):
    encoding = enc.categories_
    encoding_feature = lambda x: dict(zip(x, range(len(x))))
    encoding_full = [encoding_feature(feature_elem) for feature_elem in encoding]
    pp.pprint(encoding_full)

## 1. (40%) Correctly implement the ID3 decision tree algorithm, including the ability to handle unknown attributes (You do not need to handle real valued attributes).  
### Code Requirements/Notes:
- Use standard information gain as your basic attribute evaluation metric.  (Note that normal ID3 would usually augment information gain with gain ratio or some other mechanism to penalize statistically insignificant attribute splits. Otherwise, even with approaches like pruning below, the SSE type of overfit could still hurt us.) 
- You are welcome to create other classes and/or functions in addition to the ones provided below. (e.g. If you build out a tree structure, you might create a node class).
- It is a good idea to use a simple data set (like the lenses data or the pizza homework), which you can check by hand, to test your algorithm to make sure that it is working correctly. 

In [134]:
class DTNode():
    def __init__(self, X_data, Y_data, depth=1, is_split_node=True,
                 split_feature_index=0, feature_attribute_index=0, parent=None):

        self.X_data = X_data
        self.Y_data = Y_data
        self.depth = depth
        self.is_split_node = is_split_node # Is this node a split node?

        self.split_feature_index = split_feature_index # The index of the input features that node will split on
        self.feature_attribute_index = feature_attribute_index

        self.parent = parent # Pointer back to parent node. For computing gain

        self.value = [] # The values of each output class represented in the node
        self.children = [] # The child nodes of the self node after deciding split

        self.pred_out_class = -1

    def print_tree(self, feature_names, enc):
        pass

In [140]:
class DTClassifier(BaseEstimator,ClassifierMixin):

    def __init__(self,counts=None):
        """ Initialize class with chosen hyperparameters.
        Args:
        Optional Args (Args we think will make your life easier):
            counts: A list of Ints that tell you how many types of each feature there are
        Example:
            DT  = DTClassifier()
            or
            DT = DTClassifier(count = [2,3,2,2])
            Dataset = 
            [[0,1,0,0],
            [1,2,1,1],
            [0,1,1,0],
            [1,2,0,1],
            [0,0,1,1]]

        """
        self.counts = counts
        self.root = None
        self.n_output_classes = None
        

    def fit(self, X, y):
        """ Fit the data; Make the Decision tree

        Args:
            X (array-like): A 2D numpy array with the training data, excluding targets
            y (array-like): A 1D numpy array with the training targets

        Returns:
            self: this allows this to be chained, e.g. model.fit(X,y).predict(X_test)

        """
        self.root = DTNode(X, y)
        self.n_output_classes = len(np.unique(y))

        self._make_split(self.root)

        return self

    def predict(self, X):
        """ Predict all classes for a dataset X

        Args:
            X (array-like): A 2D numpy array with the training data, excluding targets

        Returns:
            array, shape (n_samples,)
                Predicted target values per element in X.
        """
        pass


    def score(self, X, y):
        """ Return accuracy(Classification Acc) of model on a given dataset. Must implement own score function.

        Args:
            X (array-like): A 2D numpy array with data, excluding targets
            y (array-like): A 1D numpy array of the targets 
        """
        return 0

    def print_tree(self, feature_names, enc):
        self.root.print_tree(feature_names, enc)

    def _info(self, node):
        X = node.X_data
        Y = node.Y_data

        node_info = 0
        for C in range(self.n_output_classes): # For each output class
            p_i = np.sum(np.where(Y == C, 1, 0)) / len(Y)

            if p_i != 0:
                node_info += p_i * np.log2(p_i)

        return -node_info

    def _make_split(self, node):
        print("-------------------------------------------------------")

        # Base case - When there is only one class in node
        if (len(np.unique(node.Y_data)) == 1):
            node.is_split_node = False
            node.pred_out_class = np.unique(node.Y_data)[0]
            return

        # Get the best split children and feature split index
        node.children, node.split_feature_index = self._get_best_split_children(node)


        # For each child node
            # call _make_split() on child node
        for child_node in node.children:
            self._make_split(child_node)

    def _get_best_split_children(self, node):

        # For each input feature
        X = node.X_data
        Y = node.Y_data
        len_S = len(Y)
        max_gain = 0
        best_feature_split_ind = 0
        best_split_children = None

        print("len_S: %d" % len_S)
        print("Entropy: %f" % self._info(node))
        for feature_ind in range(X.shape[1]):
            # Calculate the gain of splitting on that feature
            print("NEW FEATURE")
            contender_child_nodes = []
            info_A = 0

            for attribute_val in range(self.counts[feature_ind]): # TODO make sure this calculates correctly.
                mask = np.where(X[:, feature_ind] == attribute_val)

                X_new = X[mask]
                Y_new = Y[mask]

                # if (X_new.shape[0] == 0):
                #     break

                # pp.pprint(X_new)
                # pp.pprint(Y_new.reshape(-1, 1))

                node_new = DTNode(X_new, Y_new, depth=node.depth + 1,
                                  is_split_node=True, split_feature_index=feature_ind,
                                  feature_attribute_index=attribute_val,
                                  parent=node)
                contender_child_nodes.append(node_new)

                info_S_j = self._info(node_new)
                len_S_j = len(Y_new)
                info_A += (len_S_j / len_S) * info_S_j

                print("\t\tValue %d- %d Entropy=%f" % (attribute_val, len_S_j, info_S_j))

            gain = self._info(node) - info_A

            # print("Info: {}".format(self._info(node)))
            # print("info_A: {}".format(info_A))
            print("\t\tgain: {}".format(gain))

            if gain > max_gain:
                max_gain = gain
                best_feature_split_ind = feature_ind
                best_split_children = contender_child_nodes

        print("Split on attribute %d" % best_feature_split_ind)
        print("Best_split_children: {}".format(best_split_children))
        return best_split_children, best_feature_split_ind

## 1.1 Debug

Debug your model by training on the lenses dataset: [Debug Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/lenses.arff)

Test your model on the lenses test set: [Debug Test Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/lenses_test.arff)

Parameters:
(optional) counts = [3,2,2,2] (You should compute this when you read in the data, before fitting)

---

Expected Results: Accuracy = [0.33]

Predictions should match this file: [Lenses Predictions](https://raw.githubusercontent.com/cs472ta/CS472/master/debug_solutions/pred_lenses.csv)

*NOTE: The [Lenses Prediction](https://raw.githubusercontent.com/cs472ta/CS472/master/debug_solutions/pred_lenses.csv) uses the following encoding: soft=2, hard=0, none=1. If your encoding is different, then your output will be different, but not necessarily incorrect.*

Split Information Gains (These do not need to be in this exact order):

[0.5487949406953987, 0.7704260414863775, 0.3166890883150208, 1.0, 0.4591479170272447, 0.9182958340544894]

<!-- You should be able to get about 68% (61%-82%) predictive accuracy on the lenses data -->

Here's what your decision tree splits should look like, and the corresponding child node predictions:

Decision Tree:
<pre>
tear_prod_rate = normal:
    astigmatism = no:
        age = pre_presbyopic:
            prediction: soft
        age = presbyopic:
            spectacle_prescrip = hypermetrope:
                prediction: soft
            spectacle_prescrip = myope:
                prediction: none
        age = young:
            prediction: soft
    astigmatism = yes:
        spectacle_prescrip = hypermetrope:
            age = pre_presbyopic:
                prediction: none
            age = presbyopic:
                prediction: none
            age = young:
                prediction: hard
        spectacle_prescrip = myope:
            prediction: hard
tear_prod_rate = reduced:
    prediction: none
</pre>

In [6]:
# Load debug training and testing data 
debug_train_data = arff.loadarff('datasets/lenses.arff')
debug_trainDF = pd.DataFrame(debug_train_data[0])
for column in debug_trainDF.columns:
    debug_trainDF[column] = debug_trainDF[column] \
                         .astype(str).str \
                         .split("\'", expand=True) \
                         .iloc[:,1]

debug_test_data = arff.loadarff('datasets/lenses_test.arff')
debug_testDF = pd.DataFrame(debug_test_data[0])
for column in debug_testDF.columns:
    debug_testDF[column] = debug_testDF[column] \
                         .astype(str).str \
                         .split("\'", expand=True) \
                         .iloc[:,1]

display(debug_trainDF.head(2))
display(debug_testDF.head(2))


,age,spectacle_prescrip,astigmatism,tear_prod_rate,contact_lenses
0,young,myope,no,reduced,none
1,young,myope,no,normal,soft


,age,spectacle_prescrip,astigmatism,tear_prod_rate,contact_lenses
0,young,myope,no,reduced,soft
1,young,myope,no,reduced,hard


In [7]:
# Encode data
debug_enc = OrdinalEncoder()
debug_enc.fit(debug_trainDF)
debug_train = debug_enc.transform(debug_trainDF)
debug_test = debug_enc.transform(debug_testDF)

pp.pprint(debug_enc.categories_)
print_encodings(debug_enc)

[   array(['pre_presbyopic', 'presbyopic', 'young'], dtype=object),
    array(['hypermetrope', 'myope'], dtype=object),
    array(['no', 'yes'], dtype=object),
    array(['normal', 'reduced'], dtype=object),
    array(['hard', 'none', 'soft'], dtype=object)]
[   {'pre_presbyopic': 0, 'presbyopic': 1, 'young': 2},
    {'hypermetrope': 0, 'myope': 1},
    {'no': 0, 'yes': 1},
    {'normal': 0, 'reduced': 1},
    {'hard': 0, 'none': 1, 'soft': 2}]


In [8]:
# Save metadata
counts = debug_trainDF.nunique().to_numpy()[:-1]
features = debug_trainDF.columns

print(counts)
print(features)

[3 2 2 2]
Index(['age', 'spectacle_prescrip', 'astigmatism', 'tear_prod_rate',
       'contact_lenses'],
      dtype='object')


In [9]:
# X and Y splits
X_train = debug_train[:, :-1]
X_test = debug_test[:, -1]
Y_train = debug_train[:, -1]
Y_test = debug_test[:, -1]

In [86]:
# Train Decision Tree
dtc = DTClassifier(counts=counts)
pp.pprint(X_train)
dtc.fit(X_train, Y_train)

array([[2., 1., 0., 1.],
       [2., 1., 0., 0.],
       [2., 1., 1., 1.],
       [2., 1., 1., 0.],
       [2., 0., 0., 1.],
       [2., 0., 0., 0.],
       [2., 0., 1., 1.],
       [2., 0., 1., 0.],
       [0., 1., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 1., 1.],
       [0., 1., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 1., 1.],
       [0., 0., 1., 0.],
       [1., 1., 0., 1.],
       [1., 1., 0., 0.],
       [1., 1., 1., 1.],
       [1., 1., 1., 0.],
       [1., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 1., 1.],
       [1., 0., 1., 0.]])
NEW FEATURE
array([[0., 1., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 1., 1.],
       [0., 1., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 1., 1.],
       [0., 0., 1., 0.]])
array([[1.],
       [2.],
       [1.],
       [0.],
       [1.],
       [2.],
       [1.],
       [1.]])
array([[1., 1., 0., 1.],
       [1., 1., 0., 0.],
       [1., 1., 1., 1.],
      

DTClassifier(counts=array([3, 2, 2, 2], dtype=int64))

In [90]:
# TENNIS EXAMPLE
# Load debug training and testing data
example_train_data = arff.loadarff('datasets/examples/tennis.arff')
example_trainDF = pd.DataFrame(example_train_data[0])
for column in example_trainDF.columns:
    example_trainDF[column] = example_trainDF[column] \
                         .astype(str).str \
                         .split("\'", expand=True) \
                         .iloc[:,1]

display(example_trainDF.head())


,outlook,temperature,humidity,wind,playTennis
0,sunny,hot,high,weak,no
1,sunny,hot,high,strong,no
2,overcast,hot,high,weak,yes
3,rain,mild,high,weak,yes
4,rain,cool,normal,weak,yes


In [91]:
# Encode data
example_enc = OrdinalEncoder()
example_enc.fit(example_trainDF)
example_train = example_enc.transform(example_trainDF)
example_test = example_enc.transform(example_trainDF)

pp.pprint(example_enc.categories_)
print_encodings(example_enc)

[   array(['overcast', 'rain', 'sunny'], dtype=object),
    array(['cool', 'hot', 'mild'], dtype=object),
    array(['high', 'normal'], dtype=object),
    array(['strong', 'weak'], dtype=object),
    array(['no', 'yes'], dtype=object)]
[   {'overcast': 0, 'rain': 1, 'sunny': 2},
    {'cool': 0, 'hot': 1, 'mild': 2},
    {'high': 0, 'normal': 1},
    {'strong': 0, 'weak': 1},
    {'no': 0, 'yes': 1}]


In [92]:
# Save metadata
counts = example_trainDF.nunique().to_numpy()[:-1]
features = example_trainDF.columns

print(counts)
print(features)

[3 3 2 2]
Index(['outlook', 'temperature', 'humidity', 'wind', 'playTennis'], dtype='object')


In [93]:
# X and Y splits
X_train_example = example_train[:, :-1]
Y_train_example = example_train[:, -1]

In [141]:
# Train Decision Tree
dtc = DTClassifier(counts=counts)
pp.pprint(X_train_example)
dtc.fit(X_train_example, Y_train_example)

array([[2., 1., 0., 1.],
       [2., 1., 0., 0.],
       [0., 1., 0., 1.],
       [1., 2., 0., 1.],
       [1., 0., 1., 1.],
       [1., 0., 1., 0.],
       [0., 0., 1., 0.],
       [2., 2., 0., 1.],
       [2., 0., 1., 1.],
       [1., 2., 1., 1.],
       [2., 2., 1., 0.],
       [0., 2., 0., 0.],
       [0., 1., 1., 1.],
       [1., 2., 0., 0.]])
-------------------------------------------------------
len_S: 14
Entropy: 0.940286
NEW FEATURE
		Value 0- 4 Entropy=-0.000000
		Value 1- 5 Entropy=0.970951
		Value 2- 5 Entropy=0.970951
		gain: 0.24674981977443933
NEW FEATURE
		Value 0- 4 Entropy=0.811278
		Value 1- 4 Entropy=1.000000
		Value 2- 6 Entropy=0.918296
		gain: 0.02922256565895487
NEW FEATURE
		Value 0- 7 Entropy=0.985228
		Value 1- 7 Entropy=0.591673
		gain: 0.15183550136234159
NEW FEATURE
		Value 0- 6 Entropy=1.000000
		Value 1- 8 Entropy=0.811278
		gain: 0.04812703040826949
Split on attribute 0
Best_split_children: [<__main__.DTNode object at 0x0000023307B8A4C0>, <__main__.DTN

<ipython-input-140-0841adee9fb3>:74: RuntimeWarning: invalid value encountered in long_scalars
  p_i = np.sum(np.where(Y == C, 1, 0)) / len(Y)


DTClassifier(counts=array([3, 3, 2, 2], dtype=int64))

In [ ]:
# Predict and compute model accuracy

In [ ]:
# Print the information gain of every split you make.

In [ ]:
# Optional/Additional Debugging Dataset - Pizza Homework
# pizza_dataset = np.array([[1,2,0],[0,0,0],[0,1,1],[1,1,1],[1,0,0],[1,0,1],[0,2,1],[1,0,0],[0,2,0]])
# pizza_labels = np.array([2,0,1,2,1,2,1,1,0])

## 1.2 Evaluation

We will evaluate your model based on its performance on the zoo dataset. 

Train your model using this dataset: [Evaluation Train Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/zoo.arff)

Test your model on this dataset: [Evaluation Test Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/zoo_test.arff)

Parameters:
(optional) counts = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2] (You should compute this when you read in the data, before fitting)

---
Print out your accuracy on the evaluation test dataset.

Print out the information gain of every split you make.

In [ ]:
# Load evaluation training data


# Train Decision Tree


# Load evaluation test data


# Print out the information gain for every split you make



## 2. (20%) You will use your ID3 algorithm to induce decision trees for the cars dataset and the voting dataset.  Do not use a stopping criteria, but induce the tree as far as it can go (until classes are pure or there are no more data or attributes to split on).  
- Implement and use 10-fold Cross Validation (CV) on each data set to predict how well the models will do on novel data.  
- For each dataset, report the training and test classification accuracy for each fold and the average test accuracy. 
- As a rough sanity check, typical decision tree accuracies for these data sets are: Cars: .90-.95, Vote: .92-.95.

## 2.1 Implement 10-fold Cross Validation

In [ ]:
# Write a function that implements 10-fold cross validation

##  2.2 Cars Dataset
- Use this [Cars Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/cars.arff)
- Make a table for your K-Fold cross validation accuracies

*If you are having trouble using scipy's loadarff function (scipy.io.arff.loadarff), try:*

*pip install arff &nbsp;&nbsp;&nbsp;&nbsp;          # Install arff library*

*import arff as arf*                   

*cars = list(arf.load('cars.arff'))   &nbsp;&nbsp;&nbsp;&nbsp;# Load your downloaded dataset (!curl, etc.)*

*df = pd.DataFrame(cars)*  

*There may be additional cleaning needed*

In [ ]:
# Use 10-fold CV on Cars Dataset

# Report Training and Test Classification Accuracies

# Report Average Test Accuracy

## 2.3 Voting Dataset
- Use this [Voting Dataset with missing values](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/voting_with_missing.arff)
- Note that you will need to support unknown attributes in the voting data set. 

In [ ]:
# Load vote data 
vote_data = arff.loadarff('datasets/voting_with_missing.arff')
voteDF = pd.DataFrame(vote_data[0])
for column in voteDF.columns:
    voteDF[column] = voteDF[column] \
                         .astype(str).str \
                         .split("\'", expand=True) \
                         .iloc[:,1]

m = lambda x: "UNKNOWN" if x == "?" else x
voteDF = voteDF.applymap(m).head()
display(voteDF.head())

enc_vote = OrdinalEncoder()
enc_vote.fit(voteDF)
vote = enc_vote.transform(voteDF)
display(vote[:, :5])

X_vote_train = vote[:, :-1]
Y_vote_train = vote[:, -1]

# Used 10-fold CV on Voting Dataset

# Report Training and Test Classification Accuracies

# Report Average Test Accuracy

## 2.4 Discuss Your Results

- Summarize your results from both datasets, and discuss what you observed. 
- A fully expanded tree will often get 100% accuracy on the training set. Why does this happen and in what cases might it not?  

Discuss your results

## 3. (15%) For each of the two problems above, summarize in English what the decision tree has learned (i.e. look at the induced tree and describe what rules it has discovered to try to solve each task). 
- If the tree is very large you can just discuss a few of the more shallow attribute combinations and the most important decisions made high in the tree.

## 3.1 Discuss what the decision tree induced on the cars dataset has learned

Discussion Here

## 3.2 Discuss what the decision tree induced on the voting dataset has learned

Discussion Here

## 3.3 How did you handle unknown attributes in the voting problem? Why did you choose this approach? (Do not use the approach of just throwing out data with unknown attributes).

Discuss how you handled unknown attributes

## 4.1 (10%) Use SciKit Learn's decision tree on the voting dataset and compare your results. Try different parameters and report what parameters perform the best on the test set. 

### 4.1.1 SK Learn on Voting Dataset
- Use this [Voting Dataset with missing values](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/voting_with_missing.arff)

In [ ]:
# Use SK Learn's Decision Tree to learn the voting dataset
from sklearn import tree
from matplotlib.pyplot import figure

figure(figsize=(40, 40), dpi=300)

clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_vote_train, Y_vote_train)
tree.plot_tree(clf)
plt.show()

# Explore different parameters

# Report results

Discuss results & compare to your method's results

## 4.2 (10%) Choose a data set of your choice (not already used in this or previous labs) and use the SK decision tree to learn it. Experiment with different hyper-parameters to try to get the best results possible.

In [ ]:
# Use SciKit Learn's Decision Tree on a new dataset

# Experiment with different hyper-parameters

## 5. (5%) Visualize sklearn's decision tree for your chosen data set (using export_graphviz or another tool) and discuss what you find. If your tree is too deep to reasonably fit on one page, show only the first several levels (e.g. top 5).

In [ ]:
# Include decision tree visualization here

# Discuss what the model has learned

## 6. (optional 5% extra credit) Implement reduced error pruning to help avoid overfitting.  
- You will need to take a validation set out of your training data to do this, while still having a test set to test your final accuracy. 
- Create a table comparing your decision tree implementation's results on the cars and voting data sets with and without reduced error pruning. 
- This table should compare:
    - a) The # of nodes (including leaf nodes) and tree depth of the final decision trees 
    - b) The generalization (test set) accuracy. (For the unpruned 10-fold CV models, just use their average values in the table).